## Profile opłacalnych aplikacji w Google Play Markets i App Store
Naszym celem jest znalezienie najbardziej dochodowych aplikacji
 w sklepach Google Play i Apps Store aby pozwolić naszej firmie
 podjąć decyzje bazujące na analizie danych co do budowy najbardziej
 aplikacji opłacalnych aplikacji.
 
 Poszukujemy profilu najbardziej popularnych darmowych aplikacji, 
 które dają możliwość zarabiania na reklamach.


## Dane o aplikacjach
W Google Play Markets jak i App Store znajduję się po około 2 mln aplikacji. Zdobycie 
szczegółowych informacji na ich temat jest zasobochłonne. Zajmuje wiele czasu, jak i jest drogie.
Dlatego w niniejszej analizie wykorzystano dane ogólnie dostępne pod poniższymi  linkami:
- [Zestaw danych](https://www.kaggle.com/lava18/google-play-store-apps/home) obejmujący około 10 tysięcy aplikacji Androika z Google Play
- [Zestaw danych](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) obejmujący około 7 tysięcy aplikacji  iOS z App Store

Na początku uzyskujemy dostęp  do danych:


In [1]:
from csv import reader
# The App Store data set
open_file_a = open('AppleStore.csv')
reade_file_a = reader(open_file_a)
i_apps_list = list(reade_file_a)
i_apps_header = i_apps_list[0]
i_apps = i_apps_list[1:]
#The Google Play data set
open_file_g = open('googleplaystore.csv')
reade_file_g = reader(open_file_g)
g_apps_list = list(reade_file_g)
g_apps_header = g_apps_list[0]
g_apps = g_apps_list[1:]


Do eksploracji danych wykorzystujemy poniższą funkcję, która umożliwia wyświetlenie zawartości tabeli
zawierającej informacje na temat badanych aplikacji. Możemy wyświetlić zarówno zawartość tabeli jak i jej rozmiary (kolumny i wiersze).

In [2]:

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') 

    if rows_and_columns:
        print('Ilość wierszy:', len(dataset))
        print('Ilość kolumn:', len(dataset[0]))    

In [3]:
print(g_apps_header)
explore_data(g_apps, 4, 5, rows_and_columns=True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


Ilość wierszy: 10841
Ilość kolumn: 13


W zbiorze danych znajdują się 10841 aplikacji Androida. 
Interesujące nas kolumny zbioru to:
'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

Przyjrzyjmy się teraz danym dotyczącym Apps Store. 
W tym zbiorze znajdują się 7197 aplikacje.
Interesujące kolumny to 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'.
Objasnienie zawartości poszczególnych kolumn znajdują się w [dokumentacji](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)


In [4]:
print(i_apps_header)
explore_data(i_apps, 4, 5, rows_and_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


Ilość wierszy: 7197
Ilość kolumn: 16


Z informacji znalezionych na [forum](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) wynika, że wpis 10472 jest błędny. Porównajmy go z nagłówkiem i poprawnym wpisem.

In [5]:
print(g_apps[10472]) # błędny wpis
print(g_apps_header) # nagłówek
print(g_apps[1]) # poprawny wpis


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Z powyższego wydruku widzimy, że *Life Made WI-Fi Touchscreen Photo Frame* jako kategorię ma 1.9, a jego ocena sięga 19. Żadne z tych danych nie są poprawne. 
W konsekwencji rekord zostaje usunięty: 

In [6]:
print(len(g_apps))
del g_apps[10472] # wer run it only once :D
print(len(g_apps))

10841
10840


## Usuwanie zduplikowanych wpisów
przeglądając zbór danych pochodzących z Google Play, można zauważyć, że wpisy niektórych aplikacji pojawiają się wielokrotnie. Sprawdźmy jak ile mamy duplikatów.

In [7]:
for record in g_apps:
    if record[0] == 'Instagram':
        print(record)


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']



Poniższy kod drukuje listę zduplikowanych aplikacji.



In [8]:

def find_app_entries(data_set):
    apps = dict()
    app_entries_multip = 0
    
    for app in data_set:
        app_name = app[0]
        if app_name in apps:
            apps[app_name] += 1
            app_entries_multip += 1
        else:
            apps[app_name] = 1
    return apps, app_entries_multip


def find_multi_entries(entries):
    multi_entries = dict()
    for app, value in entries.items():
        if value > 1:
            multi_entries[app] = value
    return multi_entries


def print_multi_entries(data_set):
    apps, multi_entries = find_app_entries(data_set)
    multi_apps = find_multi_entries(apps)
    return multi_apps, multi_entries, apps



multi_apps, multi_entries, apps = print_multi_entries(g_apps)
# print('Zduplikowane aplikacje w Google Play:')
# print(multi_apps)
print("Wszystkich aplikacji jest {}.".format(len(apps)))
print('W Google Play, jest {} zduplikowanych aplikacji.'.format(len(multi_apps)))
print('Zduplikowanych wpisów jest {}. (Aplikacja może mięć wiele wpisów).'.format(multi_entries))
i_multi_apps, i_multi_entries, i_app = print_multi_entries(i_apps)
# print('Zduplikowany aplikacje w Apps Store')
# print(i_multi_apps)
print('W Apps Store, wszystkich aplikacji jest {}.'.format(len(i_app)))
print('W Apps Store, jest {} zduplikowanych aplikacji.'.format(i_multi_entries))

    

Wszystkich aplikacji jest 9659.
W Google Play, jest 798 zduplikowanych aplikacji.
Zduplikowanych wpisów jest 1181. (Aplikacja może mięć wiele wpisów).
W Apps Store, wszystkich aplikacji jest 7197.
W Apps Store, jest 0 zduplikowanych aplikacji.



Nie chcemy analizować aplikacji więcej niż jeden raz, dlatego zbędne wpisy należy usunąć. Jedną z głównych różnic we wpisach zduplikowanych aplikację widać w kolumnie czwartej która odpowiada ilości opinii. Dla naszego badania chcemy zachować te wpisy, które mają największą ilość opinii. W tym celu:
+ stworzymy słownik w których każdemu kluczowi odpowiada nazwa aplikacji, wartością zaś będzie największa ilość opinii.
    

In [9]:
max_rev = {}
for app in g_apps:
    app_name = app[0]
    app_revs = float(app[3])
    if app_name in max_rev and max_rev[app_name] < app_revs:
        max_rev[app_name] = app_revs
    else:
        max_rev[app_name] = app_revs
print('Spodziewana długość:', len(g_apps) - multi_entries)
print('Aktualna długość :', len(max_rev))

Spodziewana długość: 9659
Aktualna długość : 9659


Teraz użyjemy `max_rev` aby usunąć powielone wpisy aplikacji i zachowamy te, które maja największą ilość ocen.
+ Stworzymy dwie listy, `g_apps_clean` i `already_added`.
+ Przy pomocy pętli sprawdzimy każdą aplikację (`app`) w zbiorze Google Play (`g_apps`):
    + pobierzemy nazwę i ilość recenzji.
    + dodamy aplikację do listy `g_apps_clean` i jej nazwę do `already_added` jeśli:
        + jeśli ilość recenzji zgadza się z ilością zapisaną w `max_rev`, oraz:
        + aplikacja nie znajduje się już na liście `already_added`. 
        + musimy też wyeliminować przypadki kiedy każdy z powielonych wpisów aplikacji ma taką samą ilość recenzji, dodając warunek

In [10]:
g_apps_clean = []
already_added = []

for app in g_apps:
    app_name = app[0]
    app_revs = float(app[3])
    if max_rev[app_name] == app_revs and app_name not in already_added:
        g_apps_clean.append(app)
        already_added.append(app_name)
    

Sprawdźmy oczyszczoną listę aplikacji:


In [11]:
print(explore_data(g_apps_clean, 0, 3, True))

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Ilość wierszy: 9659
Ilość kolumn: 13
None


## Usuwanie nieanglojęzycznych aplikacji
Przeglądając zestaw danych można zauważyć, że nie wszystkie aplikacje są w języku angielskim:



In [12]:
print(i_apps[813][1])
print(i_apps[6731][1])


爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


Nie jesteśmy zainteresowani tego typu aplikacjami, gdyż są one tworzone na bardzo specyficzne rynki. Aby się ich pozbyć, usuniemy wszystkie aplikacje, których nazwy zawierają znaki spoza alfabetu łacińskiego.
Wszystkie znaki alfabetu łacińskiego kodowane są zgodznie ze standardem ASCII. Każdy znak ASCII ma przypisaną mu liczbę pomiędzy 0-127. Uzbrojeni w tą wiedzy możemy użyć wbudowanej funkcji, która znaki nazwy aplikacji. Dzięki niej dowiemy się, czy zawiera ona znaki nie należące do ASCII. Pozostawiamy możliwość akceptacji aplikacji, która posiada do 3 znaków spoza standardu, aby nasz filtr nie był zbyt restrykcyjny i akceptował znaku typu '😜'.
Poniższa funkcja wykorzystuje wbudowaną funkcję ord().


In [13]:
def is_latin(string):
    non_latin = 0
    for character in string:
        if ord(character) > 127:
            non_latin += 1
    if non_latin > 3:
        return False
    else:
        return True
print(is_latin('Instagram'))
print(is_latin('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_latin('Docs To Go™ Free Office Suite'))
print(is_latin('Instachat 😜'))


True
False
True
True


Teraz usuniemy nie anglojęzyczne aplikacje wykorzystując funkcję is_latin

In [14]:
g_apps_en = []
i_apps_en = []

for app in g_apps_clean:
    name = app[0]
    if is_latin(name):
        g_apps_en.append(app)
                         
for app in i_apps:
    name = app[1]
    if is_latin(name):
        i_apps_en.append(app)
        
explore_data(g_apps_en, 0, 3, True)
print('\n')
explore_data(i_apps_en, 0, 3, True)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Ilość wierszy: 9614
Ilość kolumn: 13


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games

Wyszukajmy teraz darmowe aplikacje.

In [15]:

android_final = []
ios_final = []
def find_free_apps(data_set, price_id):
    for app in data_set:
        price = app[price_id]
        if price == '0' or price == '0.0':
            if data_set == g_apps_en:
                android_final.append(app)
            elif data_set == i_apps_en:
                ios_final.append(app)
    
find_free_apps(g_apps_en, 7)
find_free_apps(i_apps_en, 4)

In [16]:
print(len(android_final))
print(len(ios_final))


8864
3222


Ostatecznie mamy do analizy 8864 aplikacje Google Play i 3222 z Apps Store. 

## Najpopularniejsze typy aplikacji
Tak jak to wspomnieliśmy wcześniej, naszym celem jest stworzenie profilu aplikacji, które przyciągnęłyby jak najwięcej użytkowników, gdyż naszym modelem zarabiana jest wyświetlanie reklam w darmowych aplikacjach.
Aby ograniczyć koszty operacyjne, nasza strategia polega na:
1. Zbudowaniu minimalnej aplikacji dla systemu android.
2. Jeśli aplikacja spotka się z przychylnym przyjęciem, dalszy jej rozwój.
3. Jeśli po 6 miesiącach aplikacja zacznie przynosić zysk, zbudowaniu wersji pod iOS i umieszeniu jej w App Store.

Ponieważ naszym celem jest stworzenie aplikacji zarówno pod Androida jak i iOS, powinna ona sobie radzić, na obydwu rynkach. 
Zacznijmy naszą analizę od poznania najbardziej popularnych kategorii na obu rynkach. 

In [17]:
def freq_table_prep(dataset, index):
    freq = {}
    total = 0
    for row in dataset:
        name = row[index]
        if name in freq:
            freq[name] +=1
            total +=1
        elif name not in freq:
            freq[name] = 1
            total += 1
    return freq, total
            
def freq_table(dataset, index):
    freq_percentages = {}
    freq, total = freq_table_prep(dataset, index)
    for key in freq:
        freq_percentages[key] = (freq[key] / total) * 100
    return freq_percentages
    
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


Zaczniemy naszą analizę od tabeli częstotliwości dla kolumny `prime_genre` z App Store.


In [18]:
display_table(ios_final, -5)


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


Jak widać najpopularniejsze z darmowych aplikacji to gry ("Games") jest ich ponad połowa (58,16%). Aplikacji związanych z rozrywką ("Entertainment") jest prawie osiem procent (7.88%). Następną w popularności kategoria to Zdjęcia i wideo ("Photo & Video") z prawie pięcioma procentami (4.96). Aplikacje związane z Edukacją ("Education") stanowią zaledwie 3.66% ogółu darmowych aplikacji. Jeszcze mniej jest aplikacji związanych z sieciami społecznymi ("Social Networking") jedynie 3.29%.

Jak widać najwięcej anglojęzycznych darmowych aplikacji związanych jest z ogólnie rozumianą rozrywką. Aplikacje mające praktyczne zastosowanie (nauka, zakupy, lifestyle) są raczej rzadkością.

Ilość rozrywkowych aplikacji jeszcze nic nam nie mówi o ich popularności wśród użytkowników. Może się okazać, że praktyczne aplikacje mają znacznie więcej użytkowników.

Sprawdźmy teraz jak przedstawiają się proporcje w Google Play w kolumnach Genres i Category

In [19]:
display_table(android_final, 1) # Category


FAMILY : 19.223826714801444
GAME : 9.510379061371841
TOOLS : 8.461191335740072
BUSINESS : 4.580324909747293
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.5424187725631766
SPORTS : 3.4183212996389893
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2490974729241873
HEALTH_AND_FITNESS : 3.068592057761733
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.782490974729242
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.128158844765343
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
ENTERTAINMENT : 0.8799638989169676
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 0

Na pierwszy rzut oka ilość w poszczególnych kategoriach w Google Play różni się znacznie od tego co widzieliśmy w App Store. Praktyczne aplikacje zdają się dominować. Mamy 19.22% aplikacji w kategorii rodzina ("FAMILY"), narzędzia ("TOOLS") mają 8.46% a biznes ("BUSINESS") powyżej czterech procent (4.58%). Gier ("GAME") jest tylko 9.51%. Jednak po bliższym przyjrzeniu się kategorii rodzina, wyrażnie widać, że większość z aplikacji do niej należących to gry dla dzieci.
![img](https://s3.amazonaws.com/dq-content/350/py1m8_family.png) Source: [play.google.com](https://play.google.com/store/apps/category/FAMILY?hl=en)

Podobny obraz ukazuje się nam przy analizie tabeli częstotliwości kolumny gatunek ("Genres"):

In [20]:
display_table(android_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.580324909747293
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.5424187725631766
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2490974729241873
Action : 3.1024368231046933
Health & Fitness : 3.068592057761733
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.861462093862816
Video Players & Editors : 1.782490974729242
Casual : 1.7486462093862816
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.925090252707581

Różnica pomiędzy kolumną kategoria i gatunek nie jest oczywista. Wydaje się, że tabela częstotliwości gatunku jest bardziej szczegółowa.

Z powższej analizy wiemy już, że App Store jest zdominowane przez aplikacje związane z grą i zabawą, natomiast rozkład aplikacji w Google Play jest znacznie zrównoważony. 

Przyjrzyjmy się teraz aplikacjom, które są najbardziej popularne.
## Najpopularniejsze aplikacje w gatunku w App Store 
W przeciwieństiw do Google Play, gdzie mamy podaną ilość instalacji aplikacji, App Store nie udostępnia tej informacji. Jako obejście tego problemu posłużymy się informacją o ilości opinii zawartą w kolumnie `rating_count_tot`.
Poniżej obliczymy średnią ilość użytkowników na gatunek w App Store:

In [21]:
genres_ios = freq_table(ios_final, -5)
for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)        

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


Średnio największą ilość opinii mają aplikacje nawigacji. Gatunek ten jest zdominowany prze Waze i Google Maps.

In [22]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5])


Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Podobnie sprawa się ma w przypadku aplikacji społecznościowych ("Social Networking"), gdzie pozycje dominującą mają duzi gracze jak FaceBook czy Skype.
Naszym celem jest znalezienie popularnych gatunków aplikacji. Popularność aplikacji społecznościowych i muzycznych może być wypaczone przez niezwykłą popularność kilku aplikacji. Reszta natomiast już taką popularnością się nie cieszy.
Gatunek referencje ("Reference") ma średnio 74942 opinii, ale większość z nich stanowią opinie aplikacji Bible i Dictionary.com:

In [26]:
for app in ios_final:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


Choć Biblia dominuje ranking, to ten typ aplikacji ma może mieć potencjał. Można stworzyć aplikację opartą o inny popularny tekst. Oprócz samego tekstu, aplikacja może mieć takie funkcje jak cytat dnia, nagrania, quizy, itd. Ma to o tyle sens, że jak się wydaje, rynek aplikacji związany z rozrywką zdaje się być nasycony.
 

## Najpopularniejsze aplikacje w gatunku w Google Play
W przypadku Google Play mamy dostęp do informacji na temat ilości instalacji, jest on jednak nieprecyzyjny. Większość wartości jest otwartych (1,000,000+, 1,000+ 500+, itd.).



In [28]:
display_table(android_final, 5) # the Installs columns

1,000,000+ : 15.749097472924186
100,000+ : 11.563628158844766
10,000,000+ : 10.503158844765343
10,000+ : 10.209837545126353
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


W naszym przypadku, te zaokrąglone wartością nie są dużym problemem, nie potrzebujemy dokładnych danych na temat ilości instalacji aplikacji, aby określić jakie typy aplikacji cieszą się popularnością. W analizie użyjemy niżej podanych zaokrąglonych wartości.

In [37]:
# print(g_apps_header)
# print(display_table(android_final, 1))
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ',', avg_n_installs)

ART_AND_DESIGN , 1986335.0877192982
AUTO_AND_VEHICLES , 647317.8170731707
BEAUTY , 513151.88679245283
BOOKS_AND_REFERENCE , 8767811.894736841
BUSINESS , 1704192.3399014778
COMICS , 817657.2727272727
COMMUNICATION , 38326063.197916664
DATING , 854028.8303030303
EDUCATION , 1768500.0
ENTERTAINMENT , 9146923.076923076
EVENTS , 253542.22222222222
FINANCE , 1387692.475609756
FOOD_AND_DRINK , 1924897.7363636363
HEALTH_AND_FITNESS , 4167457.3602941176
HOUSE_AND_HOME , 1331540.5616438356
LIBRARIES_AND_DEMO , 638503.734939759
LIFESTYLE , 1437816.2687861272
GAME , 12914435.883748516
FAMILY , 5180161.789906103
MEDICAL , 123064.7898089172
SOCIAL , 23253652.127118643
SHOPPING , 7036877.311557789
PHOTOGRAPHY , 17840110.40229885
SPORTS , 4274688.722772277
TRAVEL_AND_LOCAL , 13984077.710144928
TOOLS , 10801391.298666667
PERSONALIZATION , 5201482.6122448975
PRODUCTIVITY , 16772838.591304347
PARENTING , 542603.6206896552
WEATHER , 5074486.197183099
VIDEO_PLAYERS , 24790074.17721519
NEWS_AND_MAGAZINES , 

Średnio aplikacje związane z komunikacją (COMMUNICATION , 38326063) mają najwięcej instalacji.

In [38]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Messenger – Text and Video Chat for Free : 1,000,000,000+
Gmail : 1,000,000,000+
imo beta free calls and text : 100,000,000+
imo free video calls and chat : 500,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
WhatsApp Messenger : 1,000,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Hangouts : 1,000,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

Jeśli usuniemy aplikacje, które mają średnio ponad milion instalacji, śrenia instalacji jest około dziesięciokrotnie mniejsza:

In [39]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3593510.3486590036

Podobny wzorzec można dostrzec wśród aplikacji wideo, social media i aplikacjami związanymi z fotografią. Kilka bardzo popularnych aplikacji, takich jak, YouTube, Facebook, Instagram, Google Photo, dominuje rynek. Trudno rywalizować z gigantami stojącymi za tymi aplikacjami. Gry również są popularne, ale rynek wydaje się być nimi nasycony.

Spójrzmy teraz na gatunek "BOOKS_AND_REFERENCE" w Google Play. wcześniej, w przypadku Apps Store, stwierdziliśmy że nisza aplikacji związanych z książkami jest obiecująca. Naszym celem jest znalezienie profili opłacalnych aplikacji w obu sklepach. 

Średnia ilość instalacji w tym gatunku jest 8,767,811. Spójrzmy na aplikacje i ilość instalacji.

In [40]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

Aplikacje zawierają książki, referencje, słowniki, aplikacje do czytania e-booków, jak również materiały związane z programowaniem. Jest też kilka niezwykle popularnych aplikacji:

In [42]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


Tych aplikacji jest jednak niewiele, więc jest miejsce na podobne przedsięwzięcia.

Sprawdźmy jakie są aplikacje cieszące się umiarkowaną popularnością:

In [43]:
for app in android_final:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

W tym przedziale popularności dominują słowniki, czytniki więc najprawdopodobniej nie najlepszą strategią będzie konkurować z nimi.

Warto zwrócić uwagę na popularność aplikacji związanych z Koranem. Potencjalnym kierunkiem rozwoju nowych aplikacji może być skupienie się na popularnych tekstach oraz dostarczenie przydatnych rozwiązań do studiowania ich. 
## Wnioski
Analizowaliśmy sklepy z aplikacjami, Google Play i App Store w celu znalezienia niszy dla nowych aplikacji mogących przynieść dochody z prezentowania reklam. 
Naszą uwagę zwróciła nisza książek i aplikacji z nią związanych. Doszliśmy do wniosku, że aplikacja zbudowana wokół popularnego tekstu, dostarczająca dodatkowe udogodnienia, takie jak cytat dnia, quizy, forum itd, jest obiecującą drogą na rozwój w obu wyżej wymienionych serwisach.
